In [1]:
import pandas as pd
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import replicate_row,base_selection,expand_rows,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
professores_raw.columns = professores_raw.columns.astype(str)

aulas_raw = pd.read_excel('/home/cayena/Downloads/rotas_finais.xlsx')

In [5]:
aulas_simples,doub,tri=base_selection(aulas_raw)

# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = expand_rows(doub, lambda row: replicate_row(row, times=2))
aulas_triplicadas = expand_rows(tri, lambda row: replicate_row(row, times=3))

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

df_final = clean_data(df_tratado)

df_final.head()

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao,intenviso,horario_tratado
0,ANDRESSA VIP EAD - 1X,11:00:00,Vicentina,SEGUNDA,Inglês,VIP,1,NaN,ONLINE,-,0,1900-01-01 11:00:00
2,KIKO VIP P - 1X,08:00:00,Vicentina,QUINTA,Inglês,VIP - In Company,1,NaN,PRESENCIAL,BRUNO MORGILLO,0,1900-01-01 08:00:00
4,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,TERÇA,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
6,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,QUINTA,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
8,KIDS DAISY P - 2X,09:30:00,Jardim,TERÇA,Kids,Grupo,2,NaN,PRESENCIAL,CENKAY,0,1900-01-01 09:30:00


In [6]:
Ts = TeacherScheduler(df_final, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

,Professor,nome grupo
0,ALINE,ANDRESSA VIP EAD - 1X
1,DUDU,KIKO VIP P - 1X
2,ALINE,VIVIANE BENEFICIAR VIP EAD - 2X
3,ALINE,KIDS DAISY P - 2X
4,BARBARA,DELTA P - 4X
...,...,...
127,FERNANDO,BARCELONA P - 3X
128,JULIANA,GIBRALTAR P - 3X
129,ANDREY,SCOTT EAD - 3X
130,LEONARDO,NORTHAMPTOM P - 3X


In [13]:
professores_raw['08:00:00'].value_counts()

0    16
1    14
Name: 08:00:00, dtype: int64

In [23]:
df_final.loc[df_final['nome grupo'] == 'CORK EAD - 3X', 'horario'].values

array(['08:00:00', '09:00:00', '10:00:00'], dtype=object)

In [16]:
df_final[df_final['dias da semana'] =='SÁBADO']

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao,intenviso,horario_tratado
134,ROMARIO E MARILENE VIP EAD - 2X,10:00:00,Satélite,SÁBADO,Inglês,VIP,2,NaN,ONLINE,BRUNO MORGILLO,0,1900-01-01 10:00:00
709,CORK EAD - 3X,08:00:00,Jardim,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,BRUNO MORGILLO,0,1900-01-01 08:00:00
710,CORK EAD - 3X,09:00:00,Jardim,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,BRUNO MORGILLO,0,1900-01-01 09:00:00
711,CORK EAD - 3X,10:00:00,Jardim,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,BRUNO MORGILLO,0,1900-01-01 10:00:00
712,AUSTIN EAD - 3X,08:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,CLAUDIO,0,1900-01-01 08:00:00
713,AUSTIN EAD - 3X,09:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,CLAUDIO,0,1900-01-01 09:00:00
714,AUSTIN EAD - 3X,10:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,ONLINE,CLAUDIO,0,1900-01-01 10:00:00
715,BRISBANE P - 3X,08:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,PRESENCIAL,DARLENE,0,1900-01-01 08:00:00
716,BRISBANE P - 3X,09:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,PRESENCIAL,DARLENE,0,1900-01-01 09:00:00
717,BRISBANE P - 3X,10:00:00,Satélite,SÁBADO,Inglês,Grupo,3,NaN,PRESENCIAL,DARLENE,0,1900-01-01 10:00:00


In [8]:
for g in df_final['nome grupo'].unique():
    for i in professores_raw['TEACHER'].unique():
        time_class = df_final.loc[df_final['nome grupo'] == g, 'horario'].values[0] # existe turma que tem dois horarios distintos
        if professores_raw.loc[professores_raw['TEACHER'] == i, time_class].values[0] == 0:
            print(f'Professor {i} não pode dar aula no horário {time_class} para a turma {g}')

Professor ALINE não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor ANDREY não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor CARLOS não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor DANILO não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor DARLENE não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor DUDU não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor ERIK ERBEN não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor JULIANA não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor LARISSA ANDRADE não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor LEONARDO não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD - 1X
Professor LUCIANA não pode dar aula no horário 11:00:00 para a turma ANDRESSA VIP EAD 

In [ ]:
base_alocada['Professor'].value_counts()

ALINE              25
ANDREY             11
DUDU                9
BARBARA             8
ANA - MBA           8
CARLOS              7
BELLA               7
BRUNO MORGILLO      6
DANILO              5
DARLENE             5
ERIK ERBEN          5
JÉSSICA - MBA       4
CENKAY              4
CLAUDIO             4
FERNANDO            3
LEONARDO            3
JULIANA             3
LARISSA ANDRADE     2
DIEGO - ESP         2
TELMA               2
LEONEL              2
LUCIANA             1
TATIANA ERBEN       1
MADDIE              1
MARIA               1
LUCIANO             1
MARIANA             1
NATACHA             1
Name: Professor, dtype: int64